In [3]:
%%writefile get_vendor_summary.py

import pandas as pd
import logging

logging.basicConfig(
    filename="Logs/get_vendor_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

def create_vendor_summary(engine):

    logging.info("Vendor summary process started")

    query = """
    WITH FreightSummary AS (
        SELECT VendorNumber,
               SUM(Freight) AS FreightCost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),

    PurchaseSummary AS (
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price AS ActualPrice,
            pp.Volume,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases p
        JOIN purchase_prices pp
            ON p.Brand = pp.Brand
        GROUP BY
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price,
            pp.Volume
    ),

    SalesSummary AS (
        SELECT
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    )

    SELECT
        ps.*,
        ss.TotalSalesQuantity,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalExciseTax,
        fs.FreightCost,

        (ss.TotalSalesDollars - ps.TotalPurchaseDollars) AS GrossProfit,

        ((ss.TotalSalesDollars - ps.TotalPurchaseDollars)
          / NULLIF(ss.TotalSalesDollars,0))*100 AS ProfitMargin,

        ss.TotalSalesQuantity / NULLIF(ps.TotalPurchaseQuantity,0) AS StockTurnover,

        ss.TotalSalesDollars / NULLIF(ps.TotalPurchaseDollars,0) AS SalesToPurchaseRatio

    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss
        ON ps.VendorNumber = ss.VendorNo
       AND ps.Brand = ss.Brand
    LEFT JOIN FreightSummary fs
        ON ps.VendorNumber = fs.VendorNumber
    """

    df = pd.read_sql(query, engine)

    logging.info("Vendor summary created successfully")

    return df

def clean_data(df):
    """This function will clean and enhance the vendor summary"""

    # changing datatype to float
    df['Volume'] = df['Volume'].astype('float')

    # filling missing values
    df.fillna(0, inplace=True)

    # removing spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # creating new columns for better analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']

    df['ProfitMargin'] = (
        df['GrossProfit'] / df['TotalSalesDollars']
    ) * 100

    df['StockTurnover'] = (
        df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
    )

    df['SalesToPurchaseRatio'] = (
        df['TotalSalesDollars'] / df['TotalPurchaseDollars']
    )

    return df

Overwriting get_vendor_summary.py
